In [5]:
# Load environment variables
import os
import re
import pinecone
import glob
import data_import
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [6]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

In [25]:
# data_folder='../data/FEA/'
data_folder='../data/AMS/'
docs = glob.glob(data_folder+'*.pdf')   # Only get the PDFs in the directory

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

loader=PyPDFLoader(doc[-1])
doc_pages=loader.load_and_split(text_splitter)

# pages=[]
# for doc in docs:
#     loader=PyPDFLoader(doc)
#     doc_pages=loader.load_and_split(text_splitter)
#     pages.append(doc_pages)
#     print('Processed: '+doc)

Multiple definitions in dictionary at byte 0x13f4 for key /Length
Multiple definitions in dictionary at byte 0x5ad9 for key /Length
Multiple definitions in dictionary at byte 0x14a44 for key /Length
Multiple definitions in dictionary at byte 0x16e26 for key /Length
Multiple definitions in dictionary at byte 0x1a206 for key /Length
Multiple definitions in dictionary at byte 0x25e23 for key /Length
Multiple definitions in dictionary at byte 0x26aac for key /Length
Multiple definitions in dictionary at byte 0x2fa1b for key /Length
Multiple definitions in dictionary at byte 0x39302 for key /Length
Multiple definitions in dictionary at byte 0x3bc83 for key /Length
Multiple definitions in dictionary at byte 0x46ebc for key /Length
Multiple definitions in dictionary at byte 0x522b8 for key /Length
Multiple definitions in dictionary at byte 0x5d547 for key /Length
Multiple definitions in dictionary at byte 0x64fc4 for key /Length
Multiple definitions in dictionary at byte 0x6b3f2 for key /Leng

Processed: ../data/AMS\AMS_2000.pdf


Multiple definitions in dictionary at byte 0x5087 for key /Length
Multiple definitions in dictionary at byte 0x6cde for key /Length
Multiple definitions in dictionary at byte 0x12a6c for key /Length
Multiple definitions in dictionary at byte 0x1393e for key /Length
Multiple definitions in dictionary at byte 0x1f750 for key /Length
Multiple definitions in dictionary at byte 0x2d129 for key /Length
Multiple definitions in dictionary at byte 0x388e9 for key /Length
Multiple definitions in dictionary at byte 0x3dce1 for key /Length
Multiple definitions in dictionary at byte 0x46e35 for key /Length
Multiple definitions in dictionary at byte 0x51401 for key /Length
Multiple definitions in dictionary at byte 0x661fd for key /Length
Multiple definitions in dictionary at byte 0x7b119 for key /Length
Multiple definitions in dictionary at byte 0x894d0 for key /Length
Multiple definitions in dictionary at byte 0x940a0 for key /Length
Multiple definitions in dictionary at byte 0x9a4c5 for key /Leng

Processed: ../data/AMS\AMS_2001.pdf
Processed: ../data/AMS\AMS_2002.pdf
Processed: ../data/AMS\AMS_2004.pdf
Processed: ../data/AMS\AMS_2006.pdf


KeyboardInterrupt: 

In [18]:
# Tidy text up
pages_dict=[]
for page in pages:
    page.metadata['source']=os.path.basename(page.metadata['source'])   # Strip path
    page.metadata['page']=page.metadata['page']+1   # Pages are 0 based, update
    # Merge hyphenated words
    page.page_content=re.sub(r"(\w+)-\n(\w+)", r"\1\2", page.page_content)
    # Fix newlines in the middle of sentences
    page.page_content = re.sub(r"(?<!\n\s)\n(?!\s\n)", " ", page.page_content.strip())
    # Remove multiple newlines
    page.page_content = re.sub(r"\n\s*\n", "\n\n", page.page_content)

    # Format into canopy format
    dict_temp=dict(page)
    dict_temp['id']=page.metadata['source']+"_"+str(page.metadata['page'])
    dict_temp['text']=dict_temp.pop('page_content')
    dict_temp['source']=dict_temp['metadata']['source']
    pages_dict.append(dict_temp)

print(pages[0])

page_content='46th Aerospace Mechanisms Symposium    Edward A. Boesiger, Editor   Lockheed Martin Space Systems Company, Sunnyvale, California    Jonathan P. Wood, Editor   Lockheed Martin Space Systems Company, Sunnyvale, California      Insert conference information, if applicable; otherwise delete      Proceedings of a virtual symposium   Hosted by the NASA Johnson Spac e Center and Lockheed Martin Space   Sponsored and Organized by the Mechanisms Education Association    May 11-13, 2022    Click here: Press F1 key (Windows) or Help key (Mac) for help           Click here: Press F1 key (Windows) or Help National Aeronautics and   Space Administration    Johnson Space Center   Houston, Texas 77058   May 2022' metadata={'source': 'AMS_2022.pdf', 'page': 7}


In [19]:
print(pages_dict[0])

{'metadata': {'source': 'AMS_2022.pdf', 'page': 7}, 'id': 'AMS_2022.pdf_7', 'text': '46th Aerospace Mechanisms Symposium    Edward A. Boesiger, Editor   Lockheed Martin Space Systems Company, Sunnyvale, California    Jonathan P. Wood, Editor   Lockheed Martin Space Systems Company, Sunnyvale, California      Insert conference information, if applicable; otherwise delete      Proceedings of a virtual symposium   Hosted by the NASA Johnson Spac e Center and Lockheed Martin Space   Sponsored and Organized by the Mechanisms Education Association    May 11-13, 2022    Click here: Press F1 key (Windows) or Help key (Mac) for help           Click here: Press F1 key (Windows) or Help National Aeronautics and   Space Administration    Johnson Space Center   Houston, Texas 77058   May 2022', 'source': 'AMS_2022.pdf'}


In [23]:
import json

# Write the list of dictionaries to a JSON file
with open(data_folder+'ams_data.json', 'w') as f:
    json.dump(pages_dict,f,indent=2)